In [1]:
import numpy as np
import arviz as az
from line_profiler import LineProfiler
from arviz.data import load_arviz_data
from arviz.stats.stats import r2_score, hpd, _gpdfit, _gpinv
import scipy.stats as st
import numba
import math
from numpy import sin, cos
import pandas as pd

In [2]:
data_1 = np.random.randn(10_000,1000)
data_2 = np.random.randn(1_000_000)
school = load_arviz_data("centered_eight").posterior["mu"].values

In [3]:
'''********************************************hpd********************************************************************'''

'********************************************hpd********************************************************************'

In [4]:
lp = LineProfiler()
wrapper = lp(hpd)
wrapper(data_1, 0.94, True)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 4.45015 s
File: /opt/arviz/arviz/stats/stats.py
Function: hpd at line 252

Line #      Hits         Time  Per Hit   % Time  Line Contents
   252                                           def hpd(ary, credible_interval=0.94, circular=False):
   253                                               """
   254                                               Calculate highest posterior density (HPD) of array for given credible_interval.
   255                                           
   256                                               The HPD is the minimum width Bayesian credible interval (BCI). This implementation works only
   257                                               for unimodal distributions.
   258                                           
   259                                               Parameters
   260                                               ----------
   261                                               x : Numpy array
   262     

In [5]:
lp = LineProfiler()
wrapper = lp(hpd)
wrapper(data_2, 0.94, True)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.476394 s
File: /opt/arviz/arviz/stats/stats.py
Function: hpd at line 252

Line #      Hits         Time  Per Hit   % Time  Line Contents
   252                                           def hpd(ary, credible_interval=0.94, circular=False):
   253                                               """
   254                                               Calculate highest posterior density (HPD) of array for given credible_interval.
   255                                           
   256                                               The HPD is the minimum width Bayesian credible interval (BCI). This implementation works only
   257                                               for unimodal distributions.
   258                                           
   259                                               Parameters
   260                                               ----------
   261                                               x : Numpy array
   262    

In [6]:
lp = LineProfiler()
wrapper = lp(hpd)
wrapper(school, 0.94, True)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.109487 s
File: /opt/arviz/arviz/stats/stats.py
Function: hpd at line 252

Line #      Hits         Time  Per Hit   % Time  Line Contents
   252                                           def hpd(ary, credible_interval=0.94, circular=False):
   253                                               """
   254                                               Calculate highest posterior density (HPD) of array for given credible_interval.
   255                                           
   256                                               The HPD is the minimum width Bayesian credible interval (BCI). This implementation works only
   257                                               for unimodal distributions.
   258                                           
   259                                               Parameters
   260                                               ----------
   261                                               x : Numpy array
   262    

In [7]:
'''
Bottlenecks:
    1)scipy.stats.circmean
    2)numpy arctan2
'''

'\nBottlenecks:\n    1)scipy.stats.circmean\n    2)numpy arctan2\n'

In [8]:
def _circ_mean(ary, high, low):
    ary = np.asarray(ary)
    if ary.size==0:
        return np.nan, np.nan
    pi = np.pi
    angles = (ary-low)*2*pi/(high-low)
    S = sinusoidal(angles)
    C = cosine(angles)
    res = np.arctan2(S,C)
    mask = res < 0
    if mask.ndim > 0:
        res[mask] += 2*pi
    elif mask:
        res += 2*pi
    return res*(high - low)/2.0/pi + low

@numba.jit(nopython=True)
def sinusoidal(x):
    summ = 0
    x = x.flatten()
    for i in range(0,len(x)):
        summ = summ+math.sin(x[i])
    return summ
@numba.jit(nopython=True)
def cosine(x):
    summ = 0
    x = x.flatten()
    for i in range(0,len(x)):
        summ = summ+math.cos(x[i])
    return summ


In [9]:
%timeit sinusoidal(data_1)

434 ms ± 5.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%timeit np.sin(data_1)

440 ms ± 3.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%timeit sinusoidal(data_2)

37.3 ms ± 596 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%timeit np.sin(data_2)

37.9 ms ± 362 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%timeit cosine(data_1)

463 ms ± 2.55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%timeit np.cos(data_1)

464 ms ± 6.27 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%timeit cosine(data_2)

41.2 ms ± 708 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%timeit np.cos(data_2)

41.3 ms ± 1.83 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
np.allclose(_circ_mean(data_1, np.pi, -np.pi), st.circmean(data_1, np.pi, -np.pi))

True

In [18]:
np.allclose(_circ_mean(data_2, np.pi, -np.pi), st.circmean(data_2, np.pi, -np.pi))

True

In [19]:
np.allclose(_circ_mean(school, np.pi, -np.pi), st.circmean(school, np.pi, -np.pi))

True

In [20]:
%timeit _circ_mean(data_1, np.pi, -np.pi)

1.72 s ± 80.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%timeit st.circmean(data_1, np.pi, -np.pi)

1.64 s ± 94.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%timeit _circ_mean(data_2, np.pi, -np.pi)

145 ms ± 8.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
%timeit st.circmean(data_2, np.pi, -np.pi)

141 ms ± 1.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
%timeit _circ_mean(school, np.pi, -np.pi)

288 µs ± 22.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [25]:
%timeit st.circmean(school, np.pi, -np.pi)

303 µs ± 15.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [26]:
'''Performance after numba is just as good as the original circmean on large datasets and it is a bit unstable.
   On schools, the performance is much better.
   ¯\_(ツ)_/¯
   Let's see the overall perfomance of hpd
'''

"Performance after numba is just as good as the original circmean on large datasets and it is a bit unstable.\n   On schools, the performance is much better.\n   ¯\\_(ツ)_/¯\n   Let's see the overall perfomance of hpd\n"

In [27]:
def hpd(ary, credible_interval=0.94, circular=False):
    if ary.ndim > 1:
        hpd_array = np.array(
            [hpd(row, credible_interval=credible_interval, circular=circular) for row in ary.T]
        )
        return hpd_array
    # Make a copy of trace
    ary = ary.copy()
    n = len(ary)

    if circular:
        mean = _circ_mean(ary, high=np.pi, low=-np.pi)
        ary = ary - mean
        ary = np.arctan2(np.sin(ary), np.cos(ary))

    ary = np.sort(ary)
    interval_idx_inc = int(np.floor(credible_interval * n))
    n_intervals = n - interval_idx_inc
    interval_width = ary[interval_idx_inc:] - ary[:n_intervals]

    if len(interval_width) == 0:
        raise ValueError(
            "Too few elements for interval calculation. "
            "Check that credible_interval meets condition 0 =< credible_interval < 1"
        )

    min_idx = np.argmin(interval_width)
    hdi_min = ary[min_idx]
    hdi_max = ary[min_idx + interval_idx_inc]

    if circular:
        hdi_min = hdi_min + mean
        hdi_max = hdi_max + mean
        hdi_min = np.arctan2(np.sin(hdi_min), np.cos(hdi_min))
        hdi_max = np.arctan2(np.sin(hdi_max), np.cos(hdi_max))

    return np.array([hdi_min, hdi_max])

In [28]:
np.allclose(hpd(school, 0.95,True), az.stats.hpd(school, 0.95,True))

True

In [29]:
np.allclose(hpd(data_1, 0.95,True), az.stats.hpd(data_1, 0.95,True))

True

In [30]:
np.allclose(hpd(data_2, 0.95,True), az.stats.hpd(data_2, 0.95,True))

True

In [31]:
%timeit hpd(school, 0.95,True)

47.9 ms ± 2.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
%timeit az.stats.hpd(school, 0.95,True)

56.9 ms ± 3.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%timeit hpd(data_1, 0.95,True)

4.07 s ± 227 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
%timeit az.stats.hpd(data_1, 0.95,True)

4.37 s ± 102 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
%timeit hpd(data_2, 0.95,True)

482 ms ± 17.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
%timeit az.stats.hpd(data_2, 0.95,True)

491 ms ± 38.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
'''As seen above, the performance of numba is better with school dataset.
The performance with large datasets is a bit unstable.'''

'As seen above, the performance of numba is better with school dataset.\nThe performance with large datasets is a bit unstable.'

In [38]:
'''******************************************R2_SCORE***************************************************************'''

'******************************************R2_SCORE***************************************************************'

In [39]:
data_1 = np.random.randn(1000,1000)
data_2 = np.random.randn(1_000_000)
data_3 = np.random.randn(1000,1000)
data_4 = np.random.randn(1_000_000)
school = load_arviz_data("centered_eight").posterior["mu"].values
n_school = load_arviz_data("non_centered_eight").posterior["mu"].values

In [40]:
lp = LineProfiler()
wrapper = lp(r2_score)
wrapper(data_1, data_3)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.053003 s
File: /opt/arviz/arviz/stats/stats.py
Function: r2_score at line 568

Line #      Hits         Time  Per Hit   % Time  Line Contents
   568                                           def r2_score(y_true, y_pred):
   569                                               """R² for Bayesian regression models. Only valid for linear models.
   570                                           
   571                                               Parameters
   572                                               ----------
   573                                               y_true: : array-like of shape = (n_samples) or (n_samples, n_outputs)
   574                                                   Ground truth (correct) target values.
   575                                               y_pred : array-like of shape = (n_samples) or (n_samples, n_outputs)
   576                                                   Estimated target values.
   577                 

In [41]:
lp = LineProfiler()
wrapper = lp(r2_score)
wrapper(data_2, data_4)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.030574 s
File: /opt/arviz/arviz/stats/stats.py
Function: r2_score at line 568

Line #      Hits         Time  Per Hit   % Time  Line Contents
   568                                           def r2_score(y_true, y_pred):
   569                                               """R² for Bayesian regression models. Only valid for linear models.
   570                                           
   571                                               Parameters
   572                                               ----------
   573                                               y_true: : array-like of shape = (n_samples) or (n_samples, n_outputs)
   574                                                   Ground truth (correct) target values.
   575                                               y_pred : array-like of shape = (n_samples) or (n_samples, n_outputs)
   576                                                   Estimated target values.
   577                 

In [42]:
@numba.njit
def _var_1d(data):
    a,b = 0,0
    for i in data:
        a = a+i
        b = b+i*i
    return b/len(data)-((a/len(data))**2)

@numba.jit
def _var_2d(data):
    a,b = data.shape
    var = np.zeros(b)
    for i in range(0,b):
        var[i] = _var_1d(data[:,i])
    return var

In [43]:
_var_1d(data_2)

1.0001861022879104

In [44]:
np.var(data_2)

1.0001861022879106

In [45]:
np.allclose(_var_1d(data_2), np.var(data_2))

True

In [46]:
np.allclose(_var_1d(data_4), np.var(data_4))

True

In [47]:
%timeit _var_1d(data_2-data_4)

6.01 ms ± 373 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [48]:
%timeit np.var(data_2-data_4)

18.1 ms ± 1.26 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [49]:
np.allclose(_var_1d(data_2-data_4),np.var(data_2-data_4))

True

In [50]:
%timeit _var_2d(school)

49.3 µs ± 10.8 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [51]:
%timeit np.var(school,0)

68.6 µs ± 3.11 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [52]:
%timeit _var_2d(data_1)

3.28 ms ± 95.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [53]:
%timeit np.var(data_1,0)

8.2 ms ± 1.31 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [54]:
%timeit _var_2d(data_3-data_1)

7.36 ms ± 774 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [55]:
%timeit np.var(data_3-data_1,0)

19.3 ms ± 2.82 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [56]:
np.allclose(_var_2d(school), np.var(school,0))

True

In [57]:
np.allclose(_var_2d(data_1), np.var(data_1,0))

True

In [58]:
np.allclose(_var_2d(data_3), np.var(data_3,0))

True

In [59]:
np.allclose(_var_2d(data_3-data_1), np.var(data_3-data_1,0))

True

In [60]:
np.allclose(_var_2d(data_1-data_3), np.var(data_1-data_3,0))

True

In [61]:
'''Numba is doing wonders with variance. Lets inspect mean'''

'Numba is doing wonders with variance. Lets inspect mean'

In [62]:
@numba.njit
def mean_1d(data):
    summ = 0
    for i in data:
        summ = summ+i
    return summ/len(data)

In [63]:
mean_1d(data_2-data_4)

0.0006988608470254329

In [64]:
np.mean(data_2-data_4)

0.0006988608470254803

In [65]:
%timeit mean_1d(data_2-data_4)


5.46 ms ± 341 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [66]:
%timeit np.mean(data_2-data_4)

5.89 ms ± 551 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [67]:
'''Numba is not suitable here. Lets see the overall r2_score performance'''

'Numba is not suitable here. Lets see the overall r2_score performance'

In [68]:
def r2_score(y_true, y_pred):
    if y_pred.ndim == 1:
        var_y_est = _var_1d(y_pred)
        var_e = _var_1d(y_true - y_pred)
    else:
        var_y_est = _var_1d(y_pred.mean(0))
        var_e = _var_2d(y_true - y_pred)

    r_squared = var_y_est / (var_y_est + var_e)

    return pd.Series([np.mean(r_squared), np.std(r_squared)], index=["r2", "r2_std"])

In [69]:
%timeit r2_score(data_2, data_4)

8.23 ms ± 718 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [70]:
%timeit az.stats.r2_score(data_2, data_4)

25.3 ms ± 1.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [71]:
%timeit r2_score(data_1, data_3)

9.16 ms ± 443 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [72]:
%timeit az.stats.r2_score(data_1, data_3)

20.4 ms ± 726 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [73]:
%timeit r2_score(school, n_school)

1.05 ms ± 108 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [74]:
%timeit az.stats.r2_score(school, n_school)

1.2 ms ± 193 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [75]:
df_1 = r2_score(school, n_school)
df_2 = az.stats.r2_score(school, n_school)

In [76]:
df_1

r2        0.219553
r2_std    0.168670
dtype: float64

In [77]:
df_2

r2        0.219553
r2_std    0.168670
dtype: float64

In [78]:
df_1==df_2

r2        False
r2_std    False
dtype: bool

In [79]:
r2_score(data_1, data_3)

r2        0.000518
r2_std    0.000023
dtype: float64

In [80]:
az.stats.r2_score(data_1, data_3)

r2        0.000518
r2_std    0.000023
dtype: float64

In [81]:
'''I have not included the pandas test yet...will include them soon'''

'I have not included the pandas test yet...will include them soon'

In [82]:
'''It can be clearly seen that numba greatly improves the performance of r2_score.
TBH, this was the function which I thought had the least scope of improvement
Thank you mentors.'''

'It can be clearly seen that numba greatly improves the performance of r2_score.\nTBH, this was the function which I thought had the least scope of improvement\nThank you mentors.'

In [83]:
"""""""""""""""""""""""""""""""""""""""""""_gpdfit"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


'_gpdfit'

In [84]:
data = np.abs(np.sort(np.random.randn(100000)))
school = np.abs(np.sort((load_arviz_data("centered_eight").posterior["mu"].values)[1,:]))

In [85]:
lp = LineProfiler()
wrapper = lp(_gpdfit)
wrapper(data)
lp.print_stats()

/opt/arviz/arviz/stats/stats.py:520: RuntimeWarning: overflow encountered in exp
  weights = 1 / np.exp(len_scale - len_scale[:, None]).sum(axis=1)


Timer unit: 1e-06 s

Total time: 4.64491 s
File: /opt/arviz/arviz/stats/stats.py
Function: _gpdfit at line 491

Line #      Hits         Time  Per Hit   % Time  Line Contents
   491                                           def _gpdfit(ary):
   492                                               """Estimate the parameters for the Generalized Pareto Distribution (GPD).
   493                                           
   494                                               Empirical Bayes estimate for the parameters of the generalized Pareto
   495                                               distribution given the data.
   496                                           
   497                                               Parameters
   498                                               ----------
   499                                               ary : array
   500                                                   sorted 1D data array
   501                                           
   50

In [86]:
lp = LineProfiler()
wrapper = lp(_gpdfit)
wrapper(school)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.002371 s
File: /opt/arviz/arviz/stats/stats.py
Function: _gpdfit at line 491

Line #      Hits         Time  Per Hit   % Time  Line Contents
   491                                           def _gpdfit(ary):
   492                                               """Estimate the parameters for the Generalized Pareto Distribution (GPD).
   493                                           
   494                                               Empirical Bayes estimate for the parameters of the generalized Pareto
   495                                               distribution given the data.
   496                                           
   497                                               Parameters
   498                                               ----------
   499                                               ary : array
   500                                                   sorted 1D data array
   501                                           
   5

In [87]:
"""Bottleneck at 518"""

'Bottleneck at 518'

In [88]:
@numba.njit
def log_1p(data):
    log_p = np.zeros_like(data)
    for i in range(0,len(data)):
        log_p[i] = math.log1p(data[i])
    return log_p

In [89]:
np.allclose(log_1p(data), np.log1p(data))

True

In [90]:
np.allclose(log_1p(school), np.log1p(school))

True

In [91]:
%timeit log_1p(data)

2.28 ms ± 198 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [92]:
%timeit np.log1p(data)

2.16 ms ± 312 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [93]:
%timeit log_1p(school)

13.4 µs ± 787 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [94]:
%timeit np.log1p(school)

13.7 µs ± 1.47 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [95]:
'''Performance is somewhat unstable, almost half the time it speeds up the process, sometimes its performance is
while in 10% of trials the performance comes down a notch w.r.t to numpy'''

'Performance is somewhat unstable, almost half the time it speeds up the process, sometimes its performance is\nwhile in 10% of trials the performance comes down a notch w.r.t to numpy'

In [96]:
'''overall performance of gpdfit'''

'overall performance of gpdfit'

In [97]:
def _gpdfit(ary):
    prior_bs = 3
    prior_k = 10
    n = len(ary)
    m_est = 30 + int(n ** 0.5)

    b_ary = 1 - np.sqrt(m_est / (np.arange(1, m_est + 1, dtype=float) - 0.5))
    b_ary /= prior_bs * ary[int(n / 4 + 0.5) - 1]
    b_ary += 1 / ary[-1]

    k_ary = np.log1p(-b_ary[:, None] * ary).mean(axis=1) # pylint: disable=no-member Using my own log_1p here throws an exception
    len_scale = n * (np.log(-(b_ary / k_ary)) - k_ary - 1)
    weights = 1 / np.exp(len_scale - len_scale[:, None]).sum(axis=1)

    # remove negligible weights
    real_idxs = weights >= 10 * np.finfo(float).eps
    if not np.all(real_idxs):
        weights = weights[real_idxs]
        b_ary = b_ary[real_idxs]
    # normalise weights
    weights /= weights.sum()

    # posterior mean for b
    b_post = np.sum(b_ary * weights)
    # estimate for k
    k_post = log_1p(-b_post * ary).mean()  # pylint: disable=invalid-unary-operand-type,no-member
    # add prior for k_post
    k_post = (n * k_post + prior_k * 0.5) / (n + prior_k)
    sigma = -k_post / b_post
    return k_post, sigma

In [98]:
%timeit _gpdfit(data)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in exp
  del sys.path[0]


1.09 s ± 84.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [99]:
%timeit az.stats.stats._gpdfit(data)

/opt/arviz/arviz/stats/stats.py:520: RuntimeWarning: overflow encountered in exp
  weights = 1 / np.exp(len_scale - len_scale[:, None]).sum(axis=1)


1.11 s ± 76.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [100]:
%timeit _gpdfit(school)

882 µs ± 49.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [101]:
%timeit az.stats.stats._gpdfit(school)

870 µs ± 57.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [102]:
'''Similar performance on both the datasets. Up for discussion with mentors'''

'Similar performance on both the datasets. Up for discussion with mentors'

In [103]:
"""""""""""""""""""""""""""""""""""""""""""""_gpvinv"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

'_gpvinv'

In [104]:
data = np.random.randn(100000)
school = load_arviz_data("centered_eight").posterior["mu"].values[1,:]

In [105]:
lp = LineProfiler()
wrapper = lp(_gpinv)
wrapper(data, 5,2)
lp.print_stats()


Timer unit: 1e-06 s

Total time: 0.043464 s
File: /opt/arviz/arviz/stats/stats.py
Function: _gpinv at line 541

Line #      Hits         Time  Per Hit   % Time  Line Contents
   541                                           def _gpinv(probs, kappa, sigma):
   542                                               """Inverse Generalized Pareto distribution function."""
   543                                               # pylint: disable=unsupported-assignment-operation, invalid-unary-operand-type
   544         1        379.0    379.0      0.9      x = np.full_like(probs, np.nan)
   545         1          4.0      4.0      0.0      if sigma <= 0:
   546                                                   return x
   547         1       1050.0   1050.0      2.4      ok = (probs > 0) & (probs < 1)
   548         1         86.0     86.0      0.2      if np.all(ok):
   549                                                   if np.abs(kappa) < np.finfo(float).eps:
   550                            

In [106]:
lp = LineProfiler()
wrapper = lp(_gpinv)
wrapper(school, 5,2)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.000443 s
File: /opt/arviz/arviz/stats/stats.py
Function: _gpinv at line 541

Line #      Hits         Time  Per Hit   % Time  Line Contents
   541                                           def _gpinv(probs, kappa, sigma):
   542                                               """Inverse Generalized Pareto distribution function."""
   543                                               # pylint: disable=unsupported-assignment-operation, invalid-unary-operand-type
   544         1         62.0     62.0     14.0      x = np.full_like(probs, np.nan)
   545         1          5.0      5.0      1.1      if sigma <= 0:
   546                                                   return x
   547         1         55.0     55.0     12.4      ok = (probs > 0) & (probs < 1)
   548         1         85.0     85.0     19.2      if np.all(ok):
   549                                                   if np.abs(kappa) < np.finfo(float).eps:
   550                            

In [107]:
'''Log1p or expm is the bottleneck again'''

'Log1p or expm is the bottleneck again'

In [108]:
@numba.njit
def expm(data):
    expm = np.zeros_like(data)
    for i in range(0,len(data)):
        expm[i] = math.expm1(data[i])
    return expm

In [109]:
np.allclose(expm(data), np.expm1(data))

True

In [110]:
%timeit expm(data)

3.09 ms ± 196 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [111]:
%timeit np.expm1(data)

2.93 ms ± 138 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [112]:
%timeit expm(school)

15 µs ± 536 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [113]:
%timeit np.expm1(school)

14.7 µs ± 523 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [114]:
'''numpy is a bit faster
Lets see the overall performance'''

'numpy is a bit faster\nLets see the overall performance'

In [115]:
def _gpinv(probs, kappa, sigma):
    """Inverse Generalized Pareto distribution function."""
    # pylint: disable=unsupported-assignment-operation, invalid-unary-operand-type
    x = np.full_like(probs, np.nan)
    if sigma <= 0:
        return x
    ok = (probs > 0) & (probs < 1)
    if np.all(ok):
        if np.abs(kappa) < np.finfo(float).eps:
            x = -log_1p(-probs)
        else:
            x = np.expm1(-kappa * np.log1p(-probs)) / kappa
        x *= sigma
    else:
        if np.abs(kappa) < np.finfo(float).eps:
            x[ok] = -log_1p(-probs[ok])
        else:
            x[ok] = np.expm1(-kappa * np.log1p(-probs[ok])) / kappa
        x *= sigma
        x[probs == 0] = 0
        if kappa >= 0:
            x[probs == 1] = np.inf
        else:
            x[probs == 1] = -sigma / kappa
    return x

In [117]:
a = _gpinv(data,-5,2)

In [118]:
b = az.stats.stats._gpinv(data, -5, 2)

In [119]:
a

array([0.39755694,        nan,        nan, ...,        nan, 0.29756074,
              nan])

In [120]:
b


array([0.39755694,        nan,        nan, ...,        nan, 0.29756074,
              nan])

In [121]:
a-b

array([ 0., nan, nan, ..., nan,  0., nan])

In [122]:
%timeit _gpinv(data, -5, 2)

5.17 ms ± 481 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [123]:
%timeit az.stats.stats._gpinv(data, -5, 2)

4.87 ms ± 489 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [124]:
%timeit _gpinv(school, -5, 2)

72.6 µs ± 947 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [125]:
%timeit az.stats.stats._gpinv(school, -5, 2)

70.5 µs ± 1.46 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [126]:
'''numba _gpinv is similar in performance'''

'numba _gpinv is similar in performance'